In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from skimage.measure import shannon_entropy
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## First Order Features on 3000 images per class

In [106]:
dataset_path = r'F:\MS CS\1st Semester\Azheimer Classification\data_3k'


# Folder names and their corresponding labels
label_map = {
    'MildDemented': 0,
    'ModerateDemented': 1,
    'NonDemented': 2,
    'VeryMildDemented': 3
}

data = []

for folder_name, label in label_map.items():
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Extract first-order statistical features
        mean_val = np.mean(image)
        variance_val = np.var(image)
        std_val = np.std(image)
        median_val = np.median(image.flatten())
        skewness_val = skew(image.flatten())
        kurtosis_val = kurtosis(image.flatten())
        mean_abs_dev = np.mean(np.abs(image - mean_val))
        med_abs_dev = np.median(np.abs(image.flatten() - np.median(image.flatten())))

        data.append([mean_val, variance_val, std_val, median_val, skewness_val, kurtosis_val, mean_abs_dev, med_abs_dev, label])

columns = ['Mean','Variance', 'Standard Deviation', 'Median', 'Skewness', 'Kurtosis', 'Mean Absolute Deviation', 'Median Absolute Deviation', 'Label']
data_df = pd.DataFrame(data, columns=columns)
data_df.to_excel('first_order_3k.xlsx', index=False)


In [107]:
df = pd.read_excel('first_order_3k.xlsx')
df


,Mean,Variance,Standard Deviation,Median,Skewness,Kurtosis,Mean Absolute Deviation,Median Absolute Deviation,Label
0,54.735312,3633.826684,60.281230,11.0,0.873167,-0.765078,52.978060,6.0,0
1,73.443658,7076.344960,84.121014,32.0,0.608130,-1.227216,76.136920,32.0,0
2,78.717534,6761.413497,82.227815,42.0,0.471930,-1.368202,75.087423,42.0,0
3,74.248505,5895.017267,76.779016,49.0,0.513827,-1.248036,69.285903,47.0,0
4,68.004624,6279.365372,79.242447,27.0,0.728197,-1.014478,70.242814,26.0,0
...,...,...,...,...,...,...,...,...,...
11995,77.163584,7738.793553,87.970413,30.0,0.616524,-1.187401,79.524897,30.0,3
11996,130.266841,7322.002556,85.568701,91.0,0.340658,-1.632219,80.483134,41.0,3
11997,87.731266,10043.232230,100.215928,4.0,0.548918,-1.395056,92.621562,2.0,3
11998,74.200275,7064.924036,84.053102,34.0,0.596799,-1.240492,76.040888,34.0,3


In [108]:
X = df.drop(columns=['Label'])  # Features
y = df['Label']  # Labels

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [110]:
classifiers = {
    'SVM': SVC(kernel='rbf', C=1, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42)
}

for name, clf in classifiers.items():
    print(f"\n{name} Classifier:")
    
    # Train the classifier
    clf.fit(X_train, y_train)
    
    # Make predictions
    y_pred = clf.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))



SVM Classifier:
Accuracy: 0.31083333333333335

Classification Report:
               precision    recall  f1-score   support

           0       0.31      0.57      0.40       612
           1       0.67      0.00      0.01       624
           2       0.33      0.60      0.43       568
           3       0.23      0.09      0.13       596

    accuracy                           0.31      2400
   macro avg       0.38      0.32      0.24      2400
weighted avg       0.39      0.31      0.24      2400


Confusion Matrix:
 [[348   0 197  67]
 [358   2 203  61]
 [171   0 342  55]
 [247   1 294  54]]

KNN Classifier:
Accuracy: 0.34375

Classification Report:
               precision    recall  f1-score   support

           0       0.32      0.43      0.36       612
           1       0.36      0.38      0.37       624
           2       0.41      0.39      0.40       568
           3       0.27      0.17      0.21       596

    accuracy                           0.34      2400
   macro a

## 2nd Order Features on 3000 images

In [112]:
dataset_path = r'F:\MS CS\1st Semester\Azheimer Classification\data_3k'


# Folder names and their corresponding labels
label_map = {
    'MildDemented': 0,
    'ModerateDemented': 1,
    'NonDemented': 2,
    'VeryMildDemented': 3
}

data = []

for folder_name, label in label_map.items():
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        
        # Exrtract 2nd order statistical features
        
        glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        energy = graycoprops(glcm, 'energy')[0, 0]
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        entropy = shannon_entropy(image)
        idm = graycoprops(glcm, 'homogeneity')[0, 0]
        lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
        lbp_variance = np.var(lbp)

        data.append([energy, contrast, correlation, entropy, idm, lbp_variance, label])
        columns = ['Energy', 'Contrast', 'Correlation', 'Entropy', 'Idm', 'LBP', 'Label']

data_2 = pd.DataFrame(data, columns=columns)
data_2.to_excel('2nd_order_3k.xlsx', index=False)


In [113]:
df_2= pd.read_excel('2nd_order_3k.xlsx')
df_2

,Energy,Contrast,Correlation,Entropy,Idm,LBP,Label
0,0.437017,82.273022,0.988698,5.034992,0.534491,5.169138,0
1,0.414765,110.022622,0.992234,5.374151,0.509873,5.066014,0
2,0.327683,101.102298,0.992526,5.926965,0.454019,5.178979,0
3,0.354867,101.480461,0.991397,5.761734,0.465366,4.914845,0
4,0.416462,109.508768,0.991291,5.337370,0.506473,5.264572,0
...,...,...,...,...,...,...,...
11995,0.422357,137.657511,0.991115,5.398061,0.499847,5.179783,3
11996,0.405089,77.368193,0.994719,4.808894,0.578766,4.996486,3
11997,0.439179,115.904548,0.994237,4.919176,0.565840,5.036503,3
11998,0.405123,106.182095,0.992493,5.454200,0.501897,5.080425,3


In [114]:
X2= df_2.drop(columns=['Label'])  # Features
y2= df_2['Label']  # Labels

X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [115]:
classifiers = {
    'SVM': SVC(kernel='rbf', C=1, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42)
}

for name, clf in classifiers.items():
    print(f"\n{name} Classifier:")
    
    # Train the classifier
    clf.fit(X_2_train, y_2_train)
    
    # Make predictions
    y_pred = clf.predict(X_2_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_2_test, y_pred)
    print("Accuracy:", accuracy)
    print("\nClassification Report:\n", classification_report(y_2_test, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(y_2_test, y_pred))



SVM Classifier:
Accuracy: 0.2679166666666667

Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.01      0.02       612
           1       0.31      0.24      0.27       624
           2       0.26      0.81      0.39       568
           3       0.26      0.05      0.08       596

    accuracy                           0.27      2400
   macro avg       0.31      0.28      0.19      2400
weighted avg       0.31      0.27      0.19      2400


Confusion Matrix:
 [[  7 135 439  31]
 [  6 147 452  19]
 [  3  76 461  28]
 [  2 111 455  28]]

KNN Classifier:
Accuracy: 0.3458333333333333

Classification Report:
               precision    recall  f1-score   support

           0       0.29      0.38      0.33       612
           1       0.38      0.42      0.40       624
           2       0.41      0.39      0.40       568
           3       0.32      0.20      0.24       596

    accuracy                           0.35      2400


## Combining First and Second Order Features

In [116]:
dataset_path = r'F:\MS CS\1st Semester\Azheimer Classification\data_3k'


# Folder names and their corresponding labels
label_map = {
    'MildDemented': 0,
    'ModerateDemented': 1,
    'NonDemented': 2,
    'VeryMildDemented': 3
}

data = []

for folder_name, label in label_map.items():
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Extract first-order statistical features
        mean_val = np.mean(image)
        variance_val = np.var(image)
        std_val = np.std(image)
        median_val = np.median(image.flatten())
        skewness_val = skew(image.flatten())
        kurtosis_val = kurtosis(image.flatten())
        mean_abs_dev = np.mean(np.abs(image - mean_val))
        med_abs_dev = np.median(np.abs(image.flatten() - np.median(image.flatten())))
        
        # Exrtract 2nd order statistical features

        glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        energy = graycoprops(glcm, 'energy')[0, 0]
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        entropy = shannon_entropy(image)
        idm = graycoprops(glcm, 'homogeneity')[0, 0]
        lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
        lbp_variance = np.var(lbp)

        data.append([mean_val, variance_val, std_val, median_val, skewness_val, kurtosis_val, mean_abs_dev, med_abs_dev, energy, contrast, correlation, entropy, idm, lbp_variance, label])

columns = ['Mean','Variance', 'Standard Deviation', 'Median', 'Skewness', 'Kurtosis', 'Mean Absolute Deviation', 'Median Absolute Deviation', 'Energy', 'Contrast', 'Correlation', 'Entropy', 'Idm', 'LBP', 'Label']

data_df = pd.DataFrame(data, columns=columns)
data_df.to_excel('combined_3k.xlsx', index=False)


In [2]:
df = pd.read_excel('combined_3k.xlsx')
df

,Mean,Variance,Standard Deviation,Median,Skewness,Kurtosis,Mean Absolute Deviation,Median Absolute Deviation,Energy,Contrast,Correlation,Entropy,Idm,LBP,Label
0,54.735312,3633.826684,60.281230,11.0,0.873167,-0.765078,52.978060,6.0,0.437017,82.273022,0.988698,5.034992,0.534491,5.169138,0
1,73.443658,7076.344960,84.121014,32.0,0.608130,-1.227216,76.136920,32.0,0.414765,110.022622,0.992234,5.374151,0.509873,5.066014,0
2,78.717534,6761.413497,82.227815,42.0,0.471930,-1.368202,75.087423,42.0,0.327683,101.102298,0.992526,5.926965,0.454019,5.178979,0
3,74.248505,5895.017267,76.779016,49.0,0.513827,-1.248036,69.285903,47.0,0.354867,101.480461,0.991397,5.761734,0.465366,4.914845,0
4,68.004624,6279.365372,79.242447,27.0,0.728197,-1.014478,70.242814,26.0,0.416462,109.508768,0.991291,5.337370,0.506473,5.264572,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,77.163584,7738.793553,87.970413,30.0,0.616524,-1.187401,79.524897,30.0,0.422357,137.657511,0.991115,5.398061,0.499847,5.179783,3
11996,130.266841,7322.002556,85.568701,91.0,0.340658,-1.632219,80.483134,41.0,0.405089,77.368193,0.994719,4.808894,0.578766,4.996486,3
11997,87.731266,10043.232230,100.215928,4.0,0.548918,-1.395056,92.621562,2.0,0.439179,115.904548,0.994237,4.919176,0.565840,5.036503,3
11998,74.200275,7064.924036,84.053102,34.0,0.596799,-1.240492,76.040888,34.0,0.405123,106.182095,0.992493,5.454200,0.501897,5.080425,3


In [118]:
X = df.drop(columns=['Label'])  # Features
y = df['Label']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [119]:
classifiers = {
    'SVM': SVC(kernel='rbf', C=1, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42)
}

for name, clf in classifiers.items():
    print(f"\n{name} Classifier:")
    
    # Train the classifier
    clf.fit(X_train, y_train)
    
    # Make predictions
    y_pred = clf.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))



SVM Classifier:
Accuracy: 0.31166666666666665

Classification Report:
               precision    recall  f1-score   support

           0       0.31      0.57      0.40       612
           1       0.50      0.00      0.00       624
           2       0.33      0.61      0.43       568
           3       0.23      0.09      0.13       596

    accuracy                           0.31      2400
   macro avg       0.34      0.32      0.24      2400
weighted avg       0.34      0.31      0.24      2400


Confusion Matrix:
 [[348   0 198  66]
 [359   1 205  59]
 [170   0 347  51]
 [247   1 296  52]]

KNN Classifier:
Accuracy: 0.36

Classification Report:
               precision    recall  f1-score   support

           0       0.34      0.44      0.38       612
           1       0.38      0.41      0.39       624
           2       0.44      0.42      0.43       568
           3       0.26      0.17      0.20       596

    accuracy                           0.36      2400
   macro avg 

## Now extracting features only for Moderate and Non Dementiated
---

## First Order Features

In [120]:
dataset_path = r'F:\MS CS\1st Semester\Azheimer Classification\two_classes_3k'


# Folder names and their corresponding labels
label_map = {
    'ModerateDemented': 0,
    'NonDemented': 1,
}

data = []

for folder_name, label in label_map.items():
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Extract first-order statistical features
        mean_val = np.mean(image)
        variance_val = np.var(image)
        std_val = np.std(image)
        median_val = np.median(image.flatten())
        skewness_val = skew(image.flatten())
        kurtosis_val = kurtosis(image.flatten())
        mean_abs_dev = np.mean(np.abs(image - mean_val))
        med_abs_dev = np.median(np.abs(image.flatten() - np.median(image.flatten())))

        data.append([mean_val, variance_val, std_val, median_val, skewness_val, kurtosis_val, mean_abs_dev, med_abs_dev, label])

columns = ['Mean','Variance', 'Standard Deviation', 'Median', 'Skewness', 'Kurtosis', 'Mean Absolute Deviation', 'Median Absolute Deviation', 'Label']
data_df = pd.DataFrame(data, columns=columns)
data_df.to_excel('two_classes_first_order_3k.xlsx', index=False)


In [121]:
df = pd.read_excel('two_classes_first_order_3k.xlsx')
df

,Mean,Variance,Standard Deviation,Median,Skewness,Kurtosis,Mean Absolute Deviation,Median Absolute Deviation,Label
0,72.950933,7425.848238,86.173362,31.0,0.726425,-1.015362,76.695123,31.0,0
1,82.393893,4392.084684,66.272805,32.0,0.788855,-1.016406,58.791629,4.0,0
2,84.676399,6788.072040,82.389757,71.0,0.357875,-1.395940,74.354088,71.0,0
3,70.483299,6401.592594,80.009953,35.0,0.617048,-1.179041,71.997265,35.0,0
4,68.444814,6330.368447,79.563613,5.0,0.765340,-0.982587,70.974853,1.0,0
...,...,...,...,...,...,...,...,...,...
5995,94.658024,7443.660097,86.276649,103.0,0.127652,-1.581385,79.413149,101.0,1
5996,67.512855,5995.566082,77.431041,31.0,0.658878,-1.090769,69.231177,31.0,1
5997,96.625857,7946.381960,89.142481,107.0,0.149296,-1.571269,81.703401,106.0,1
5998,83.070811,8014.532474,89.523921,44.0,0.405285,-1.478734,83.175332,44.0,1


In [122]:
X = df.drop(columns=['Label'])  # Features
y = df['Label']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [124]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8216666666666667

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.86      0.83       613
           1       0.84      0.79      0.81       587

    accuracy                           0.82      1200
   macro avg       0.82      0.82      0.82      1200
weighted avg       0.82      0.82      0.82      1200


Confusion Matrix:
 [[525  88]
 [126 461]]


## 2nd Order Features

In [125]:
dataset_path = r'F:\MS CS\1st Semester\Azheimer Classification\two_classes_3k'


# Folder names and their corresponding labels
label_map = {
    'ModerateDemented': 0,
    'NonDemented': 1,
}

data = []

for folder_name, label in label_map.items():
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        
        # Exrtract 2nd order statistical features
        
        glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        energy = graycoprops(glcm, 'energy')[0, 0]
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        entropy = shannon_entropy(image)
        idm = graycoprops(glcm, 'homogeneity')[0, 0]
        lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
        lbp_variance = np.var(lbp)

        data.append([energy, contrast, correlation, entropy, idm, lbp_variance, label])
        columns = ['Energy', 'Contrast', 'Correlation', 'Entropy', 'Idm', 'LBP', 'Label']

data_df = pd.DataFrame(data, columns=columns)
data_df.to_excel('two_classes_2nd_order_3k.xlsx', index=False)


In [126]:
df = pd.read_excel('two_classes_2nd_order_3k.xlsx')
df

,Energy,Contrast,Correlation,Entropy,Idm,LBP,Label
0,0.431744,138.978099,0.990654,5.314018,0.509669,5.215329,0
1,0.438719,59.282511,0.993262,4.748322,0.593685,4.910794,0
2,0.308876,98.766176,0.992723,6.096612,0.426652,4.964118,0
3,0.409382,100.309978,0.992173,5.403963,0.500029,5.136254,0
4,0.452901,91.523543,0.992782,5.070750,0.542606,4.859707,0
...,...,...,...,...,...,...,...
5995,0.300098,97.652026,0.993434,6.081695,0.427352,5.039618,1
5996,0.408638,113.586443,0.990537,5.393945,0.503930,5.371528,1
5997,0.309994,76.771140,0.995166,5.988073,0.446812,5.154775,1
5998,0.435149,85.010330,0.994700,5.170395,0.527049,5.302010,1


In [127]:
X = df.drop(columns=['Label'])  # Features
y = df['Label']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [128]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8233333333333334

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.89      0.84       613
           1       0.86      0.76      0.81       587

    accuracy                           0.82      1200
   macro avg       0.83      0.82      0.82      1200
weighted avg       0.83      0.82      0.82      1200


Confusion Matrix:
 [[543  70]
 [142 445]]


## Combining first and second order features

In [129]:
dataset_path = r'F:\MS CS\1st Semester\Azheimer Classification\two_classes_3k'


# Folder names and their corresponding labels
label_map = {
    'ModerateDemented': 0,
    'NonDemented': 1,
}

data = []

for folder_name, label in label_map.items():
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist.")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Extract first-order statistical features
        mean_val = np.mean(image)
        variance_val = np.var(image)
        std_val = np.std(image)
        median_val = np.median(image.flatten())
        skewness_val = skew(image.flatten())
        kurtosis_val = kurtosis(image.flatten())
        mean_abs_dev = np.mean(np.abs(image - mean_val))
        med_abs_dev = np.median(np.abs(image.flatten() - np.median(image.flatten())))
        
        # Exrtract 2nd order statistical features

        glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        energy = graycoprops(glcm, 'energy')[0, 0]
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        entropy = shannon_entropy(image)
        idm = graycoprops(glcm, 'homogeneity')[0, 0]
        lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
        lbp_variance = np.var(lbp)

        data.append([mean_val, variance_val, std_val, median_val, skewness_val, kurtosis_val, mean_abs_dev, med_abs_dev, energy, contrast, correlation, entropy, idm, lbp_variance, label])

columns = ['Mean','Variance', 'Standard Deviation', 'Median', 'Skewness', 'Kurtosis', 'Mean Absolute Deviation', 'Median Absolute Deviation', 'Energy', 'Contrast', 'Correlation', 'Entropy', 'Idm', 'LBP', 'Label']

data_df = pd.DataFrame(data, columns=columns)
data_df.to_excel('two_classes_combined_3k.xlsx', index=False)


In [130]:
df = pd.read_excel('two_classes_combined_3k.xlsx')
df

,Mean,Variance,Standard Deviation,Median,Skewness,Kurtosis,Mean Absolute Deviation,Median Absolute Deviation,Energy,Contrast,Correlation,Entropy,Idm,LBP,Label
0,72.950933,7425.848238,86.173362,31.0,0.726425,-1.015362,76.695123,31.0,0.431744,138.978099,0.990654,5.314018,0.509669,5.215329,0
1,82.393893,4392.084684,66.272805,32.0,0.788855,-1.016406,58.791629,4.0,0.438719,59.282511,0.993262,4.748322,0.593685,4.910794,0
2,84.676399,6788.072040,82.389757,71.0,0.357875,-1.395940,74.354088,71.0,0.308876,98.766176,0.992723,6.096612,0.426652,4.964118,0
3,70.483299,6401.592594,80.009953,35.0,0.617048,-1.179041,71.997265,35.0,0.409382,100.309978,0.992173,5.403963,0.500029,5.136254,0
4,68.444814,6330.368447,79.563613,5.0,0.765340,-0.982587,70.974853,1.0,0.452901,91.523543,0.992782,5.070750,0.542606,4.859707,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,94.658024,7443.660097,86.276649,103.0,0.127652,-1.581385,79.413149,101.0,0.300098,97.652026,0.993434,6.081695,0.427352,5.039618,1
5996,67.512855,5995.566082,77.431041,31.0,0.658878,-1.090769,69.231177,31.0,0.408638,113.586443,0.990537,5.393945,0.503930,5.371528,1
5997,96.625857,7946.381960,89.142481,107.0,0.149296,-1.571269,81.703401,106.0,0.309994,76.771140,0.995166,5.988073,0.446812,5.154775,1
5998,83.070811,8014.532474,89.523921,44.0,0.405285,-1.478734,83.175332,44.0,0.435149,85.010330,0.994700,5.170395,0.527049,5.302010,1


In [131]:
X = df.drop(columns=['Label'])  # Features
y = df['Label']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8725

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.92      0.88       613
           1       0.91      0.82      0.86       587

    accuracy                           0.87      1200
   macro avg       0.88      0.87      0.87      1200
weighted avg       0.88      0.87      0.87      1200


Confusion Matrix:
 [[566  47]
 [106 481]]
